# Impoart

In [1]:
import import_ipynb
import torch
import torchvision
import matplotlib.pyplot as plt
import numpy as np

import bbox_visualizer as bbv

from MaskRCNN_VOC import *


import mh_utils as MH

from MaskRCNN_VOC import VOC2012_MaskRCNN_InstanceSegmentation as Task
from MaskRCNN_VOC import VOC2012_MaskRCNN_InstanceSegmentation_Dataset as TaskDataset
from MaskRCNN_VOC import VOC2012_MaskRCNN_InstanceSegmentation_DataLoader as TaskDataloader


importing Jupyter notebook from MaskRCNN_VOC.ipynb
importing Jupyter notebook from mh_utils.ipynb


/opt/conda/lib/python3.8/site-packages/nbformat/__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


In [3]:
np.random.seed(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
t_dataset, t_dataloader, v_dataset, v_dataloader = Task.get_dataset_and_loader(10)
model = Task.get_model("result/banch")

for iou_threshold in [0.3, 0.5, 0.6]
    map = Task.map_eval(model, device, data_loader, iou_threshold = iou_threshold)
    print(f"map({iou_threshold}): {map}")


In [4]:
IOU_THRESHOLD = 0.5

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

dataset = TaskDataset(root = "/home/VOCdevkit/VOC2012", image_set = "val", cropping = False)
data_loader = TaskDataloader(dataset, batch_size = 10)

model = Task.get_model("models/no_expending(8)")




/opt/conda/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_V2_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 145/145 [00:51<00:00,  2.84it/s]


0.31593098542893056

100%|██████████| 145/145 [00:47<00:00,  3.06it/s]


0.35887285647100803

In [5]:
model = Task.get_model()
model.load_state_dict(torch.load("models/boundary_reinforcement(2th)"))

CLASS_NUM = Task.CLASS_NUM
MASK_THRESHOLD = 0.5
SCORE_THRESHOLDS = Task.SCORE_THRESHOLDS

Task.map_eval(model, device, data_loader, 
    class_num = CLASS_NUM, 
    mask_threshold = MASK_THRESHOLD,
    score_threshold = SCORE_THRESHOLDS)

100%|██████████| 145/145 [00:47<00:00,  3.03it/s]


0.35565320588827226